## Script for analysing the velocities in Sebasitan Viscaino Bay

In [ ]:
from xmitgcm import open_mdsdataset
from MITgcmutils import rdmds
from MITgcmutils import mds
from MITgcmutils import diagnostics
import xmitgcm 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as dat
from matplotlib.ticker import FormatStrFormatter
from matplotlib.gridspec import GridSpec
from matplotlib import cm
from matplotlib.lines import Line2D
from mpl_toolkits import mplot3d
import xarray as xr
import cmocean
import time 
import pylab as pl
import IPython
from IPython import display
from matplotlib.animation import FuncAnimation
from mpl_toolkits.axes_grid1 import make_axes_locatable
import ffmpeg
from matplotlib import animation
from math import radians, cos, sin, asin, sqrt
import netCDF4 as nc
from scipy.io import savemat
import scipy as sc
import scipy.io as sio
import scipy.signal as sig
from statistics import mean
import datetime
import warnings
warnings.filterwarnings("ignore")

In [ ]:
params = {'font.size': 22,
          'figure.figsize': (30, 30),
         'font.family':'serif'}
pl.rcParams.update(params)


# Loading values

In [ ]:
def loadNetCDFACs(varname,alldepths):
    dsw=[]
    dsn=[]
    i=0
    for i in np.arange(0,8,1):
        if alldepths==0:
            pathn='/media/amelia/Trillian/SVB/exp06_512x612x100_ORL/01_noSVB_febTS/'+ str(varname)+'ACnoSVBdep55'+ str(2+i)+'_'+ str(3+i) +'all.nc'
            pathw='/media/amelia/Trillian/SVB/exp06_512x612x100_ORL_SVB/01_SVB_febTS/'+ str(varname)+'ACwithSVBdep55'+ str(2+i)+'_'+ str(3+i) +'all.nc'
        else:
            pathn='/media/amelia/Trillian/SVB/exp06_512x612x100_ORL/01_noSVB_febTS/'+ str(varname)+'ACnoSVB'+ str(2+i)+'_'+ str(3+i) +'diagonal.nc'
            pathw='/media/amelia/Trillian/SVB/exp06_512x612x100_ORL_SVB/01_SVB_febTS/'+ str(varname)+'ACwithSVB'+ str(2+i)+'_'+ str(3+i) +'diagonal.nc'
        dswin  = xr.open_dataset(pathw)
        dsnin = xr.open_dataset(pathn)

        dsw.append(dswin)
        dsn.append(dsnin)

    return dsw, dsn

def loadWVEL(dsw,dsn,alldepths):
    if alldepths==1:
        rangedep=np.arange(0,71,10)
        var23=dsw[0].Ww[:,rangedep,:].values
        var34=dsw[1].Ww[:,rangedep,:].values
        var45=dsw[2].Ww[:,rangedep,:].values
        var56=dsw[3].Ww[:,rangedep,:].values
        var67=dsw[4].Ww[:,rangedep,:].values
        var78=dsw[5].Ww[:,rangedep,:].values
        var89=dsw[6].Ww[:,rangedep,:].values
        var910=dsw[7].Ww[:,rangedep,:].values

        Ww=np.concatenate((var23, var34, var45, var56,var67,var78,var89, var910), axis=0)# , ,var910), axis=0) 

        var23n=dsn[0].Wn[:,rangedep,:].values
        var34n=dsn[1].Wn[:,rangedep,:].values
        var45n=dsn[2].Wn[:,rangedep,:].values
        var56n=dsn[3].Wn[:,rangedep,:].values                
        var67n=dsn[4].Wn[:,rangedep,:].values
        var78n=dsn[5].Wn[:,rangedep,:].values
        var89n=dsn[6].Wn[:,rangedep,:].values
        var910n=dsn[7].Wn[:,rangedep,:].values

        Wn=np.concatenate((var23n, var34n, var45n, var56n,var67n,var78n,var89n, var910n), axis=0) #, var910n), axis=0) 
    else:
        var23=dsw[0].Ww.values
        var34=dsw[1].Ww.values
        var45=dsw[2].Ww.values
        var56=dsw[3].Ww.values
        var67=dsw[4].Ww.values
        var78=dsw[5].Ww.values
        var89=dsw[6].Ww.values
        var910=dsw[7].Ww.values

        Ww=np.concatenate((var23, var34, var45, var56,var67,var78,var89, var910), axis=0)# , ,var910), axis=0) 

        var23n=dsn[0].Wn.values
        var34n=dsn[1].Wn.values
        var45n=dsn[2].Wn.values
        var56n=dsn[3].Wn.values                
        var67n=dsn[4].Wn.values
        var78n=dsn[5].Wn.values
        var89n=dsn[6].Wn.values
        var910n=dsn[7].Wn.values

        Wn=np.concatenate((var23n, var34n, var45n, var56n,var67n,var78n,var89n, var910n), axis=0) #, var910n), axis=0) 
   
    time23=dsw[0].time.values.astype(int)
    time34=dsw[1].time.values.astype(int)
    time45=dsw[2].time.values.astype(int)
    time56=dsw[3].time.values.astype(int)
    time67=dsw[4].time.values.astype(int)
    time78=dsw[5].time.values.astype(int)
    time89=dsw[6].time.values.astype(int)
    time910=dsw[7].time.values.astype(int)
    
    Time=np.concatenate((time23, time34, time45, time56,time67, time78,time89, time910), axis=0)#, time910), axis=0)
    
    times=Time*1e-9
      
    return Ww,Wn,times

def loadVVEL(dsw,dsn):
    var23=dsw[0].Vw.values
    var34=dsw[1].Vw.values
    var45=dsw[2].Vw.values
    var56=dsw[3].Vw.values
    var67=dsw[4].Vw.values
    var78=dsw[5].Vw.values
    var89=dsw[6].Vw.values
    var910=dsw[7].Vw.values
    
    Vw=np.concatenate((var23, var34, var45, var56,var67, var78,var89, var910), axis=0)#, var910), axis=0) 
    
    var23n=dsn[0].Vn.values
    var34n=dsn[1].Vn.values
    var45n=dsn[2].Vn.values
    var56n=dsn[3].Vn.values               
    var67n=dsn[4].Vn.values
    var78n=dsn[5].Vn.values
    var89n=dsn[6].Vn.values
    var910n=dsn[7].Vn.values
    
    Vn=np.concatenate((var23n, var34n, var45n, var56n,var67n, var78n,var89n, var910n), axis=0) #, var910n), axis=0) 
    
    return Vw,Vn
    
    
def loadUVEL(dsw,dsn):
    var23=dsw[0].Uw.values
    var34=dsw[1].Uw.values
    var45=dsw[2].Uw.values
    var56=dsw[3].Uw.values
    var67=dsw[4].Uw.values
    var78=dsw[5].Uw.values
    var89=dsw[6].Uw.values
    var910=dsw[7].Uw.values
    
    Uw=np.concatenate((var23, var34, var45, var56,var67, var78,var89, var910), axis=0)#), axis=0) 
    
    var23n=dsn[0].Un.values
    var34n=dsn[1].Un.values
    var45n=dsn[2].Un.values
    var56n=dsn[3].Un.values               
    var67n=dsn[4].Un.values
    var78n=dsn[5].Un.values
    var89n=dsn[6].Un.values
    var910n=dsn[7].Un.values
    
    Un=np.concatenate((var23n, var34n, var45n, var56n,var67n, var78n,var89n, var910n), axis=0) #, var910n), axis=0) 
    
    return Uw,Un        

## Loading values along the coast

In [ ]:
#Filtering
def butter_lowpass(cutoff, fs, order=3):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = sig.butter(order, normal_cutoff, btype='bandpass', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=3):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = sig.filtfilt(b, a, data)
    return y

# Loading depth over time values
def loadvertical():
    ds=[]
    
    hej=[58,65, 111, 125, 130, 205,220, 267,278, 299, 360,405]
    for l in np.arange(len(hej)):
        
        path=f'WVELvertical{hej[l]}.nc'
        dsin = xr.open_dataset(path)

        ds.append(dsin)


    return ds

#Loading values that only come for 1 depth, so  points along the coast for example
def loadMITgcmVelocities(dep,detrend,filtering):
    
    alldepths=0
    
    dww,dwn=loadNetCDFACs('WVEL',alldepths)
    dvw,dvn=loadNetCDFACs('VVEL',alldepths)
    duw,dun=loadNetCDFACs('UVEL',alldepths)

    Ww,Wn,TIME=loadWVEL(dww,dwn,alldepths)
    Vw,Vn=loadVVEL(dvw,dvn)
    Uw,Un=loadUVEL(duw,dun)

    W=Ww-Wn
    V=Vw-Vn
    U=Uw-Un
    
    i=0
    varname='PHIHYD'
    pathw='/media/amelia/Trillian/SVB/exp06_512x612x100_ORL_SVB/01b_SVB_febTS/'+ str(varname)+'withSVB'+ str(2+i)+'_'+ str(3+i) +'.nc'
        
    dswdep  = xr.open_dataset(pathw)
    depth=dswdep.Depth
    LON=dswdep.XC-360
    LAT=dswdep.YC

    Z=dswdep.Z

    hFacCw = dswdep.hFacC
    hFacCusew=hFacCw.values

    hfa = np.ma.masked_values(hFacCusew, 0)
    mask = np.ma.getmask(hfa)

    ind30=220 #30° N
    ind307=220+50 #30.7° N
    ind314=220+100 #31.4° N
    ind32=220+150 #32° N

    
    dist=dww[0].x 
    lon_ac=dww[0].lonAC
    lat_ac=dww[0].latAC
    
    dist30=dist[np.where(lat_ac==ind30)]
    dist307=dist[np.where(lat_ac==ind307)]
    dist314=dist[np.where(lat_ac==ind314)]
    dist32=dist[np.where(lat_ac==ind32)]
    
    dsvert=loadvertical()

    if detrend==1:
        Wdif=sig.detrend(W,0)
        Vdif=sig.detrend(V,0)
        Udif=sig.detrend(U,0)
    if filtering==1:
        Wfilt=np.zeros(np.shape(Wdif))
        Vfilt=np.zeros(np.shape(Vdif))
        Ufilt=np.zeros(np.shape(Udif))
        
        order = 3
        fs = 1/1200       
        cutoff =np.array([1/432000, 1/43200])
        for d in np.arange(np.size(Wdif,1)):
            data = Wdif[:,d]
            # Filtering and plotting
            Wfilt[:,d] = butter_lowpass_filter(data, cutoff, fs, order)

        for d in np.arange(np.size(Vdif,1)):
            data = Vdif[:,d]
            # Filtering and plotting
            Vfilt[:,d] = butter_lowpass_filter(data, cutoff, fs, order)
    
        for d in np.arange(np.size(Udif,1)):
            data = Udif[:,d]
            # Filtering and plotting
            Ufilt[:,d] = butter_lowpass_filter(data, cutoff, fs, order)    
        
    else:
        Wdif=0
        Wfilt=0
        Vdif=0
        Vfilt=0
        Udif=0
        Ufilt=0
    
    hej=[58, 85, 205, 227]
    p=0
    distcross=np.zeros((4,100))
    depcross=np.zeros((4,100))
    for i in hej:
        p=p+1
        ind=lat_ac[i]
        lon=LAT[lat_ac[i]]

        coast=np.where(mask[0,ind,:]==False)

        LONin = dswdep.XC[coast]-360
        distance=cos((lon*np.pi)/180)*111*-1*(LONin-LONin[-1])

        indlonpre=np.where(distance<=100)
        indlon=np.flip(indlonpre[0])
        #actual distance from the coast
        distpre=distance[indlon]
        distcross[p-1,:len(distpre)]=distpre
        #depth 
        depcross[p-1,:len(depth[ind,indlon].values)]=depth[ind,indlon].values
            
            
    return(W,V,U,mask,TIME,LON,LAT,Z,depth,dist,lon_ac,lat_ac,Wdif,
           Wfilt,Vdif,Vfilt,Udif,Ufilt,ind30,ind307,ind314,ind32,
          dist30,dist307,dist314,dist32,dsvert,depcross,distcross) 

In [ ]:
dep=55
detrend=1
filtering=1

W,V,U,mask,TIME,LON,LAT,Z,depth,dist,lon_ac,lat_ac,Wdif,Wfilt,Vdif,Vfilt,Udif,Ufilt,ind30,ind307,ind314,ind32,\
dist30,dist307,dist314,dist32,dsvert,depcross,distcross =loadMITgcmVelocities(dep,detrend,filtering)

### Saving the values in a smaller file for the values along the coast

In [ ]:
FILENAME='coastVEL.nc'
ds = xr.Dataset({'Vdif': (("time","dist"),Vdif),
                 'Vfilt':(("time","dist"),Vfilt),
                 'Wdif': (("time","dist"),Wdif),
                 'Wfilt':(("time","dist"),Wfilt),
                 'Udif': (("time","dist"),Udif),
                 'Ufilt':(("time","dist"),Ufilt)
                    },
                coords ={
                    "lon_ac" : lon_ac,
                    "lat_ac" : lat_ac,
                    "time": TIME,
                    "dist":dist.values
                },
                )
ds.to_netcdf(FILENAME)

# Changes in velocity along the coast 

In [ ]:
alldepths=0
dww,dwn=loadNetCDFACs('WVEL',alldepths)
dvw,dvn=loadNetCDFACs('VVEL',alldepths)
duw,dun=loadNetCDFACs('UVEL',alldepths)

In [ ]:
alldepths=0
Ww,Wn,time=loadWVEL(dww,dwn,alldepths)
Vw,Vn=loadVVEL(dvw,dvn)
Uw,Un=loadUVEL(duw,dun)

In [ ]:
W=Ww-Wn
V=Vw-Vn
U=Uw-Un

In [ ]:
Wdif=sig.detrend(W,0)
Vdif=sig.detrend(V,0)
Udif=sig.detrend(U,0)

In [ ]:
i=0
varname='DYNVARS'
pathw='/media/amelia/Trillian/SVB/exp06_512x612x100_ORL_SVB/01_SVB_febTS/'+ str(varname)+'withSVB'+ str(2+i)+'_'+ str(3+i) +'.nc'
        
dswdep  = xr.open_dataset(pathw)
depth=dswdep.Depth
LON=dswdep.XC-360
LAT=dswdep.YC

Z=dswdep.Z

hFacCw = dswdep.hFacC
hFacCusew=hFacCw.values

hfa = np.ma.masked_values(hFacCusew, 0)
maskw = np.ma.getmask(hfa)

ind30=220 #30° N
ind307=220+50 #30.7° N
ind314=220+100 #31.4° N
ind32=220+150 #32° N


# Defining Butterworth filter

In [ ]:
def butter_lowpass(cutoff, fs, order=3):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = sig.butter(order, normal_cutoff, btype='bandpass', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=3):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = sig.filtfilt(b, a, data)
    return y

# When looking at all points along the coast

#### Loading variables

#### Visualising filtering

In [ ]:
params = {'font.size': 16,
          'figure.figsize': (9, 9),
         'font.family':'sans'}
pl.rcParams.update(params)


In [ ]:
# Setting standard filter requirements.
order = 3
fs = 1/1200       
cutoff =np.array([1/432000, 1/43200])

b, a = butter_lowpass(cutoff, fs, order)

# Plotting the frequency response.
w, h = sig.freqz(b, a, worN=8000)
ax=plt.subplot(2, 1, 1)
plt.plot((0.5*fs*w/np.pi)*1e4, np.abs(h), 'b')
plt.plot(cutoff*1e4, [0.5*np.sqrt(2),0.5*np.sqrt(2)], 'ko')
ax.text(0.95,0.92,'(a)',horizontalalignment='center',transform=ax.transAxes)
plt.axvline(cutoff[0]*1e4, color='k')
plt.axvline(cutoff[1]*1e4, color='k')
#plt.xlim(0, 0.2*fs)

plt.title("Bandpass Filter Frequency Response")
plt.xlabel('Frequency [Hz]')
plt.grid()


# Creating the data for filteration
t=TIME*60
d=42
data = W[:,d*1]

# Filtering and plotting
y = butter_lowpass_filter(data, cutoff, fs, order)

ax1=plt.subplot(2, 1, 2)
ax1.plot(t/(60*60*24), data, 'b-', label='data')
ax1.plot(t/(60*60*24), y, 'g-', linewidth=2, label='filtered data')
plt.xlabel('Time [sec]')
plt.grid()
plt.legend(loc='lower right')
plt.title("Vertical velocity")
ax1.text(0.95,0.92,'(b)',horizontalalignment='center',transform=ax1.transAxes)
plt.subplots_adjust(hspace=0.35)
plt.show()

plt.tight_layout()

#### Filtering

In [ ]:
order = 5
fs = 1/1200       
cutoff =np.array([1/432000, 1/43200])

Wfilt=np.zeros(np.shape(Wdif))

for d in np.arange(np.size(Wdif,1)):
    data = Wdif[:,d]
    # Filtering and plotting
    Wfilt[:,d] = butter_lowpass_filter(data, cutoff, fs, order)
    
Vfilt=np.zeros(np.shape(Vdif))

for d in np.arange(np.size(Vdif,1)):
    data = Vdif[:,d]
    # Filtering and plotting
    Vfilt[:,d] = butter_lowpass_filter(data, cutoff, fs, order)
    
Ufilt=np.zeros(np.shape(Udif))
for d in np.arange(np.size(Udif,1)):
    data = Udif[:,d]
    # Filtering and plotting
    Ufilt[:,d] = butter_lowpass_filter(data, cutoff, fs, order)

#### Finding the derivative to find the period

In [ ]:
Wderiv=(Wfilt[:-1,100]-Wfilt[1:,100])/(TIME[:-1]-TIME[1:])
asign=np.sign(Wderiv)
signchange = np.where(((np.roll(asign, 1) - asign) != 0))
TIMEmax=TIME[signchange]
print((TIMEmax[2:]-TIMEmax[:-2])/(60*24))


fig,ax=plt.subplots(2)
ax[0].plot((Wfilt[:-1,300]-Wfilt[1:,300])/(TIME[:-1]-TIME[1:]))
ax[1].plot(TIME[:],Wfilt[:,0])

#### Showing effect of the filter

In [ ]:
params = {'font.size': 28,
          'figure.figsize': (30, 30),
         'font.family':'serif'}
pl.rcParams.update(params)


In [ ]:
fig,ax=plt.subplots(6,2)

d=39
dd=6
xlab='Time (days)'
ylab='Vertical velocity (m/s)'
distance=dist.values
VAL=Wfilt[:,:]
VAL2=W[:,:]
ylim=[-0.00001, 0.00001]
ylim2=[-0.0000075, 0.0000075]
ylim3=[-0.000005, 0.000005]
ylim4=[-0.0000025, 0.0000025]
ylim5=[-0.000001, 0.000001]

plt.setp(ax, xticks=[2880, 4320, 5760, 7200, 8640, 10080, 11520, 12960, 14400,], xticklabels=[2, 3, 4, 5, 6, 7, 8, 9, 10])

ax[0,0].plot(TIME, VAL2[:,0],'r')
ax[0,0].plot(TIME, VAL[:,0])
ax[0,0].set(xlabel=xlab, ylabel=ylab)
ax[0,0].set_title('%1.1f km from the bay' %distance[d*0])
ax[0,0].set_ylim([-np.nanmax(VAL[:,0]),np.nanmax(VAL[:,0])])

ax[0,1].plot(TIME, VAL2[:,d],'r')
ax[0,1].plot(TIME, VAL[:,d])
ax[0,1].set(xlabel=xlab, ylabel=ylab)
ax[0,1].set_title('%1.1f km from the bay' %distance[d*1])
ax[0,1].set_ylim([-np.nanmax(VAL[:,d]),np.nanmax(VAL[:,d])])

ax[1,0].plot(TIME, VAL2[:,d*2],'r')
ax[1,0].plot(TIME, VAL[:,d*2])
ax[1,0].set(xlabel=xlab, ylabel=ylab)
ax[1,0].set_title('%1.1f km from the bay' %distance[d*2])
ax[1,0].set_ylim([-np.nanmax(VAL[:,d*2]),np.nanmax(VAL[:,d*2])])

ax[1,1].plot(TIME, VAL2[:,d*3],'r')
ax[1,1].plot(TIME, VAL[:,d*3])
ax[1,1].set(xlabel=xlab, ylabel=ylab)
ax[1,1].set_title('%1.1f km from the bay' %distance[d*3])
ax[1,1].set_ylim([-np.nanmax(VAL[:,d*3]),np.nanmax(VAL[:,d*3])])

ax[2,0].plot(TIME, VAL2[:,d*4],'r')
ax[2,0].plot(TIME, VAL[:,d*4])
ax[2,0].set(xlabel=xlab, ylabel=ylab)
ax[2,0].set_title('%1.1f km from the bay' %distance[d*4])
ax[2,0].set_ylim([-np.nanmax(VAL[:,d*4]),np.nanmax(VAL[:,d*4])])

ax[2,1].plot(TIME, VAL2[:,d*5],'r')
ax[2,1].plot(TIME, VAL[:,d*5])
ax[2,1].set(xlabel=xlab, ylabel=ylab)
ax[2,1].set_title('%1.1f km from the bay' %distance[d*5])
ax[2,1].set_ylim([-np.nanmax(VAL[:,d*5]),np.nanmax(VAL[:,d*5])])

ax[3,0].plot(TIME, VAL2[:,d*6],'r')
ax[3,0].plot(TIME, VAL[:,d*6])
ax[3,0].set(xlabel=xlab, ylabel=ylab)
ax[3,0].set_title('%1.1f km from the bay' %distance[d*6])
ax[3,0].set_ylim([-np.nanmax(VAL[:,d*6]),np.nanmax(VAL[:,d*6])])

ax[3,1].plot(TIME, VAL2[:,d*7],'r')
ax[3,1].plot(TIME, VAL[:,d*7])
ax[3,1].set(xlabel=xlab, ylabel=ylab)
ax[3,1].set_title('%1.1f km from the bay' %distance[d*7])
ax[3,1].set_ylim([-np.nanmax(VAL[:,d*7]),np.nanmax(VAL[:,d*7])])

ax[4,0].plot(TIME, VAL2[:,d*8],'r')
ax[4,0].plot(TIME, VAL[:,d*8])
ax[4,0].set(xlabel=xlab, ylabel=ylab)
ax[4,0].set_title('%1.1f km from the bay' %distance[d*8])
ax[4,0].set_ylim([-np.nanmax(VAL[:,d*8]),np.nanmax(VAL[:,d*8])])

ax[4,1].plot(TIME, VAL2[:,d*9],'r')
ax[4,1].plot(TIME, VAL[:,d*9])
ax[4,1].set(xlabel=xlab, ylabel=ylab)
ax[4,1].set_title('%1.1f km from the bay' %distance[d*9])
ax[4,1].set_ylim([-np.nanmax(VAL[:,d*9]),np.nanmax(VAL[:,d*9])])

ax[5,0].plot(TIME, VAL2[:,d*10],'r')
ax[5,0].plot(TIME, VAL[:,d*10])
ax[5,0].set(xlabel=xlab, ylabel=ylab)
ax[5,0].set_title('%1.1f km from the bay' %distance[d*10])
ax[5,0].set_ylim([-np.nanmax(VAL[:,d*10]),np.nanmax(VAL[:,d*10])])

ax[5,1].plot(TIME, VAL2[:,d*11],'r')
ax[5,1].plot(TIME, VAL[:,d*11])
ax[5,1].set(xlabel=xlab, ylabel=ylab)
ax[5,1].set_title('%1.1f km from the bay' %distance[d*11])
ax[5,1].set_ylim([-np.nanmax(VAL[:,d*11]),np.nanmax(VAL[:,d*11])])

fig.tight_layout()

#for i in np.arange(0,12,1):

 #   Wderiv=(Wfilt[:-1,dd,d*i]-Wfilt[1:,dd,d*i])/(TIME[:-1]-TIME[1:])

  #  asign=np.sign(Wderiv)
   # signchange = np.where(((np.roll(asign, 1) - asign) != 0))

   # TIMEmax=TIME[signchange]
   # perpre=(TIMEmax[2:]-TIMEmax[:-2])/(60*24)
    #per=mean(perpre[perpre>=0.8])
    
    #print('%1.1f km from the bay' %distance[d*i])
    #print(f'Mean period after removing anomalies is {per:.3f} days ')


##### Define how to make Hovmoller plot

In [ ]:
params = {'font.size': 30,
          'figure.figsize': (30, 30),
         'font.family':'serif'}
pl.rcParams.update(params)

In [ ]:
def plot_HOVMÖLLER(ax,LON,TIME,VAL,title,ctitle,vmin,vmax,fig,lat,lon,lines,cbarall):
    
    xlab='Time [days]'
    ylab='Distance [km]' #'Depth [m]'

    if lines ==1:
        ind30=220 #30° N
        ind307=220+50 #30.7° N
        ind314=220+100 #31.4° N
        ind32=220+150 #32° N

        dist30=LON[np.where(lat==ind30)]
        dist307=LON[np.where(lat>=ind307)]
        dist314=LON[np.where(lat==ind314)]
        dist32=LON[np.where(lat>=ind32)]
        distsvb=LON[45]
        distendwaves=LON[108]
    
        ax.axhline(y=distsvb,color='k',linewidth=5)
        ax.axhline(y=distendwaves,color='k',linewidth=5)
        ax.annotate('Where SVB is',xy=(0.93,0.4), xytext=(0.7, 0.15),xycoords='axes fraction')

    
    ax.set(xlabel=xlab, ylabel=ylab)
    ax.set_xticks([2880, 4320, 5760, 7200, 8640, 10080, 11520, 12960, 14400])
    ax.set_xticklabels([2, 3, 4, 5, 6, 7, 8, 9, 10])
    ax.set_title(title)


    cax = ax.pcolormesh(TIME,LON,np.transpose(VAL),cmap=cmocean.cm.balance,vmin=vmin,vmax=vmax) 
    
    if cbarall==1:
    ##FOR THE SAME COLORBAR FOR ALL OF THE PLOTS
        cbar_ax = fig.add_axes([1, 0.15, 0.03, 0.7])
        fig.colorbar(cax, cax=cbar_ax)
        cbar_ax.set_ylabel(ctitle)
    else:
    ##FOR A COLORBAR FOR EACH PLOT
        cbar = plt.colorbar(cax)
        cbar.set_label(ctitle)
    


#### Hovmöller plot of vertical velocity in progress of filtering

In [ ]:
fig = plt.figure(figsize=(30, 30))
gs = GridSpec(nrows=3, ncols=1, height_ratios=[1, 1,1])

vmin=-0.000003
vmax=0.000003
lines=0
cbarall=1

latmult=lat_ac.values
latarray=np.ones_like(W)*latmult[None,:]
lonmult=lon_ac.values
lonarray=np.ones_like(W)*lonmult[None,:]
maskpre=np.logical_or(lonarray >= 137, latarray >= 552)
mask=maskpre==False

ax0 = fig.add_subplot(gs[0, 0])
plot_HOVMÖLLER(ax0,dist,TIME,W,'Vertical velocity raw data',
               'Vertical velocity [m/s]',vmin,vmax,fig,lat_ac,lon_ac,lines,cbarall)
ax1 = fig.add_subplot(gs[1, 0])
plot_HOVMÖLLER(ax1,dist,TIME,Wdif,'Vertical velocity detrended',
               'Vertical velocity [m/s]',vmin,vmax,fig,lat_ac,lon_ac,lines,cbarall)
ax2 = fig.add_subplot(gs[2, 0])
plot_HOVMÖLLER(ax2,dist,TIME,Wfilt,'Vertical velocity filtered',
               'Vertical velocity [m/s]',vmin,vmax,fig,lat_ac,lon_ac,lines,cbarall)


fig.tight_layout()
maskmaybe=np.logical_or(lon_ac.values>=190, lat_ac.values>=552)

### Calculating shelf width

In [ ]:
params = {'font.size': 28,
          'figure.figsize': (30, 15),
         'font.family':'serif'}
pl.rcParams.update(params)


In [ ]:
dsdepth = xr.open_dataset("/media/amelia/Trillian/SVB/exp06_512x612x100_ORL_SVB/01_SVB_febTS/ETAwithSVBACall.nc")        

lon_coast=dsdepth.lonAC
lat_coast=dsdepth.latAC

In [ ]:
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in kilometers between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
    return c * r

In [ ]:
depthsmooth=np.zeros((len(LAT),len(LON)))
for jj in np.arange(0,len(LAT),1):
    for ii in np.arange(5,len(LON[:-5]),1):
        depthsmooth[jj,ii]=np.mean(depth[jj,ii-5:ii+5])
depthsmooth=depthsmooth[:,10:-10]

In [ ]:
lon_of_shelf=np.zeros(np.size(lat_coast[10:-10]))
dep=np.zeros(np.size(lat_coast[10:-10]))
p=0
for jj in lat_coast[10:-10]:
    p=p+1
    d=depth[jj,depth[jj,:]<=400].values
    maxi=np.argmax(abs(d[1:]-d[:-1]))
    location=np.where(depth[jj,:]==d[maxi])
    lon_of_shelf[p-1]=location[-1][-1]
    dep[p-1]=depth[jj,location[-1][-1]]

In [ ]:
dist_array = np.zeros(len(lon_ac))

p=0
for ii,jj,ll,kk in zip(lon_ac,lat_ac,lat_coast[7:-7],lon_coast[7:-7]):
    lat1 = LAT[jj]
    lon1 = LON[ii]
    lat2 = LAT[ll]
    lon2 = LON[kk]
    p=p+1
    dist_array[p-1]=  haversine(lat1, lon1, lat2, lon2)



In [ ]:
fig,ax=plt.subplots()
ax.plot(dist_array,dist)
ax.set_xlabel('Shelf width [km]')
ax.set_ylabel('Distance along the coast [km]')

In [ ]:
fig,ax=plt.subplots()
plt.plot(depth[lat_ac,lon_ac])
ax.set_xlabel('Distance along the coast [km]')
ax.set_ylabel('Depth [m]')

## Vertical Hovmöller (change hovmoller to be over depth not distance)

In [ ]:
params = {'font.size': 28,
          'figure.figsize': (30, 30),
         'font.family':'serif'}
pl.rcParams.update(params)

In [ ]:
def loadvertical():
    ds=[]
    
    hej=[58,65, 111, 125, 130, 205,220, 267,278, 299, 360,405]
    for l in np.arange(len(hej)):
        
        path=f'WVELvertical{hej[l]}.nc'
        dsin = xr.open_dataset(path)

        ds.append(dsin)


    return ds

In [ ]:
ds=loadvertical()

In [ ]:
hej=[58,65, 111, 125, 130, 205,220, 267,278, 299, 360,405]

In [ ]:
fig = plt.figure()
gs = GridSpec(nrows=4, ncols=3)#, height_ratios=[1,1])
Wmin=-0.000003
Wmax=0.000003
lines=0
cbarall=1
n=0
p=0
for l in np.arange(len(hej)):
    p=p+1
    if l!=0:
        
        if l % 3 == 0:
            p=0
            p=p+1
            n=n+1
    ax = fig.add_subplot(gs[n, p-1])
    plot_HOVMÖLLER(ax,Z[:55],TIME,(ds[l].Ww[:,:55].values-ds[l].Wn[:,:55].values),f'{dist[hej[l]].values:.1f} km from the bay',
                   'Vertical velocity [m/s]',Wmin,Wmax,fig,lat_ac,lon_ac,lines,cbarall)
fig.tight_layout()

In [ ]:
fig = plt.figure()
gs = GridSpec(nrows=1, ncols=2)#, height_ratios=[1,1])

lines=0
cbarall=0
hej=[58,65, 111, 125, 130, 205,220, 267,278, 299, 360,405]
Wmin=-0.000003
Wmax=0.000003
markers=Line2D.filled_markers
markers=np.delete(markers,np.arange(2,5,1))
colors=['b','g','r','k','m','c','y','brown','lime','fuchsia','darkorange','white']

ax0 = fig.add_subplot(gs[0, 0])
plot_HOVMÖLLER(ax0,dist,TIME,Wfilt,'',
               'Vertical velocity [m/s]',Wmin,Wmax,fig,lat_ac,lon_ac,lines,cbarall)
p=0
for i in hej:
    p=p+1
    ax0.axhline(y=dist[i],color=colors[p-1])
    ax0.plot(TIME[288],dist[i],'o', 
            markersize=18, color=colors[p-1],marker=markers[p-1])

ax3 = fig.add_subplot(gs[0,1])

pc = ax3.plot(dist_array,dist)
p=0
for i in hej:
    p=p+1
    ax3.axhline(y=dist[i],color=colors[p-1])
    ax3.plot(80,dist[i],'o', 
            markersize=18, color=colors[p-1],marker=markers[p-1])
ax3.set_xlabel('Shelf width [km]')
ax3.set_ylabel('Distance [km]')
#ax3.set_aspect(1)
fig.tight_layout()


fig2 = plt.figure()
gs = GridSpec(nrows=4, ncols=3)#, height_ratios=[1,1])
Wmin=-0.000003
Wmax=0.000003
lines=0
cbarall=1
n=0
p=0
for l in np.arange(len(hej)):
    p=p+1
    if l!=0:
        
        if l % 3 == 0:
            p=0
            p=p+1
            n=n+1
    ax = fig2.add_subplot(gs[n, p-1])
    plot_HOVMÖLLER(ax,Z[:55],TIME,(dsvert[l].Ww[:,:55].values-ds[l].Wn[:,:55].values),f'{dist[hej[l]].values} km from the bay',
                   'Vertical velocity [m/s]',Wmin,Wmax,fig2,lat_ac,lon_ac,lines,cbarall)
fig2.tight_layout()


In [ ]:
fig = plt.figure(figsize=(40, 30))
ax3 = plt.axes()

ax3.set_facecolor('tan')
pc = ax3.contourf(LON,LAT,np.ma.masked_array(depth, mask=maskw[0,:,:]),50,
                 vmin=0, vmax=5000, cmap=cmocean.cm.deep)#, extend='max')
cb = plt.colorbar(pc, extend='max',label='depth / m')
#cn = ax3.contour(LON,LAT,depth, colors=['0.3','0.6'], 
 #               levels=[250,500])
#ax3.axhline(y=LAT[518],color='k')
#ax3.annotate('800 km from the bay',xy=(340,350), xytext=(340, 350+50),xycoords='axes points',fontsize=10)



for ii,jj in zip(lon_ac,lat_ac):

    ax3.plot(LON[ii-1],LAT[jj-1],'o', 
            markersize=14, color='r')


#ax3.plot(LON[108],LAT[506],'o', 
 #       markersize=4, color='b')

#ax.plot(LON[lon_fix[]],LAT[lat_fix[0]],'o', 
 #       markersize=4, color='b')
#ax.plot(LON[lon_fix[0]],LAT[lat_fix[0]],'o', 
 #       markersize=4, color='b')


cb.set_label('Depth [m]')
ax3.set_xlabel('Lon [°]')
ax3.set_ylabel('Lat [°]')
ax3.set_xlim(238-360, 246-360)
ax3.set_ylim(27,35.3)
ax3.set_aspect(1)
fig.tight_layout()

In [ ]:
fig = plt.figure()
gs = GridSpec(nrows=1, ncols=2)#, height_ratios=[1,1])

lines=0
cbarall=0
#hej=[58, 85, 111, 125, 130, 205, 227,267, 299, 360,405]
hej=[58, 85, 205, 227]
Wmin=-0.000003
Wmax=0.000003
markers=Line2D.filled_markers
markers=np.delete(markers,np.arange(2,5,1))
colors=['b','g','m','k','r','c','y','brown','lime','fuchsia','beige']

ax0 = fig.add_subplot(gs[0, 0])
plot_HOVMÖLLER(ax0,dist,TIME,Wfilt,'Vertical velocity [m/s]',
               'Vertical velocity [m/s]',Wmin,Wmax,fig,lat_ac,lon_ac,lines,cbarall)
ax0.text(0.96,0.96,'(a)',horizontalalignment='center',transform=ax0.transAxes)
p=0
for i in hej:
    p=p+1
    ax0.axhline(y=dist[i],color=colors[p-1])
    ax0.plot(TIME[288],dist[i],'o', 
            markersize=18, color=colors[p-1],marker=markers[p-1])
    ax0.annotate(f'{dist[i].values:.2f} km from bay',(TIME[305],dist[i]+10),color='k') 

ax3 = fig.add_subplot(gs[0,1])
ax3.set_facecolor('tan')
pc = ax3.contourf(LON,LAT,np.ma.masked_array(depth, mask=maskw[0,:,:]),50,
                 vmin=0, vmax=5000, cmap=cmocean.cm.deep)#, extend='max')
cb = plt.colorbar(pc, extend='max',label='depth / m')
cn = ax3.contour(LON,LAT,depth, colors=['0.3','0.6'], 
                levels=[250,500])

ax3.text(0.96,0.96,'(b)',horizontalalignment='center',transform=ax3.transAxes)

for ii,jj in zip(lon_ac,lat_ac):

    ax3.plot(LON[ii-1],LAT[jj-1],'o', 
            markersize=8, color='r')
p=0
for i in hej:
    p=p+1
    ax3.axhline(y=LAT[lat_ac[i]],color=colors[p-1],marker=markers[p-1])
    ax3.annotate(f'{LAT[lat_ac[i]].values:.2f}°N',(LON[410],LAT[lat_ac[i]]+0.01),color='k') 
p=0
for ii,jj in zip(lon_ac[hej],lat_ac[hej]):
    p=p+1
    ax3.plot(LON[ii-1],LAT[jj-1],marker=markers[p-1], 
            markersize=18, color=colors[p-1])
    


#for ii,jj in zip(lon_of_shelf.astype(int),lat_coast):

 #   ax3.plot(LON[ii],LAT[jj],'o', 
  #          markersize=4, color='r')
    


cb.set_label('Depth [m]')
ax3.set_xlabel('Lon [°]')
ax3.set_ylabel('Lat [°]')
ax3.set_xlim(238-360, 246-360)
ax3.set_ylim(27,35.3)
#ax3.set_aspect(1)
fig.tight_layout()


In [ ]:
params = {'font.size': 27,
          'figure.figsize': (25, 14),
         'font.family':'serif'}
pl.rcParams.update(params)

In [ ]:
fig = plt.figure()
gs = GridSpec(nrows=2, ncols=2, width_ratios=[1,0.5])

lines=0
cbarall=0
#hej=[58, 85, 111, 125, 130, 205, 227,267, 299, 360,405]
hej=[58, 85, 205, 227]
Wmin=-0.000003
Wmax=0.000003
markers=Line2D.filled_markers
markers=np.delete(markers,np.arange(2,5,1))
colors=['b','g','r','k','m','c','y','brown','lime','fuchsia','beige']

ax0 = fig.add_subplot(gs[0, 1])
for i in np.arange(0,4,1):
    ax0.plot(distcross[i,depcross[i,:]!=0],-depcross[i,depcross[i,:]!=0], label=f'{dist[hej[i]].values:.0f} km', color=colors[i])

ax0.text(0.96,0.92,'(b)',horizontalalignment='center',transform=ax0.transAxes)

ax3 = fig.add_subplot(gs[0:2,0])
lines=0
cbarall=0
#hej=[58, 85, 111, 125, 130, 205, 227,267, 299, 360,405]
hej=[58, 85, 205, 227]
Wmin=-0.000003
Wmax=0.000003
markers=Line2D.filled_markers
markers=np.delete(markers,np.arange(2,5,1))
colors=['b','g','r','k','m','c','y','brown','lime','fuchsia','beige']

plot_HOVMÖLLER(ax3,dist,TIME,Wfilt,'Vertical velocity [m/s]',
               'Vertical velocity [m/s]',Wmin,Wmax,fig,lat_ac,lon_ac,lines,cbarall)
ax3.text(0.96,0.96,'(a)',horizontalalignment='center',transform=ax3.transAxes)
p=0
for i in hej:
    p=p+1
    ax3.axhline(y=dist[i],color=colors[p-1])
    ax3.plot(TIME[288],dist[i],'o', 
            markersize=18, color=colors[p-1],marker=markers[p-1])
    ax3.annotate(f'{dist[i].values:.0f} km',(TIME[500],dist[i]+10),color='k') 

fig.tight_layout()

ax0.set_xlabel('Distance from coast [km]')
ax0.set_ylabel('Depth [m]')
ax0.legend()

#### Hovmöller plot of all the velocities along with a map showing where the data has been taken from

In [ ]:
fig = plt.figure(figsize=(40, 40))
gs = GridSpec(nrows=2, ncols=3, height_ratios=[1,1.5])

lines=0
cbarall=0

Wmin=-0.003
Wmax=0.003
Vmin=-0.1
Vmax=0.1
Umin=-0.1
Umax=0.1

ax0 = fig.add_subplot(gs[0, 0])
plot_HOVMÖLLER(ax0,dist,TIME,Wfilt*1000,'Vertical velocity',
               'Vertical velocity [mm/s]',Wmin,Wmax,fig,lat_ac,lon_ac,lines,cbarall)
ax1 = fig.add_subplot(gs[0, 1])
plot_HOVMÖLLER(ax1,dist,TIME,Vfilt*1000,'Alongshore velocity',
               'Alongshore velocity [mm/s]',Vmin,Vmax,fig,lat_ac,lon_ac,lines,cbarall) 
ax2 = fig.add_subplot(gs[0, 2])
plot_HOVMÖLLER(ax2,dist,TIME,Ufilt*1000,'Crosshore velocity',
               'Crosshore velocity [mm/s]',Umin,Umax,fig,lat_ac,lon_ac,lines,cbarall)

ax3 = fig.add_subplot(gs[1,0:3])
ax3.set_facecolor('tan')
pc = ax3.contourf(LON,LAT,np.ma.masked_array(depth, mask=maskw[0,:,:]),50,
                 vmin=0, vmax=5000, cmap=cmocean.cm.deep)#, extend='max')
cb = plt.colorbar(pc, extend='max',label='depth / m')
cn = ax3.contour(LON,LAT,depth, colors=['0.3','0.6'], 
                levels=[250,500])
#ax3.axhline(y=LAT[518],color='k')
#ax3.annotate('800 km from the bay',xy=(340,350), xytext=(340, 350+50),xycoords='axes points',fontsize=10)



for ii,jj in zip(lon_ac,lat_ac):


    ax3.plot(LON[ii-1],LAT[jj-1],'o', 
            markersize=10, color='r')


ax3.plot(LON[108],LAT[506],'o', 
        markersize=4, color='b')

#ax.plot(LON[lon_ac[]],LAT[lat_ac[0]],'o', 
 #       markersize=4, color='b')
#ax.plot(LON[lon_ac[0]],LAT[lat_ac[0]],'o', 
 #       markersize=4, color='b')


cb.set_label('Depth (m)')
ax3.set_xlabel('Lon [°]')
ax3.set_ylabel('Lat [°]')
ax3.set_xlim(238-360, 246-360)
ax3.set_ylim(27,35.3)
ax3.set_aspect(1)
fig.tight_layout()

#### Calculate phase speed with lagged correlation

In [ ]:
index30=np.where(lat_ac==ind30)
index307=np.where(lat_ac==ind307)
index314=np.where(lat_ac==ind314)
index32=np.where(lat_ac==ind32)


##### For the latitudes in Brink

In [ ]:
lag=np.zeros(4)
p=0

xlab='Lag'
ylab='Correlation'

for i in np.arange(1,5,1):
    p=p+1
    index=[0,46,99,149,207]
    in1=Wfilt[:,index[i]]
    in2=Wfilt[:,index[i-1]]
    out=sig.correlate(in1, in2)
    lags = sig.correlation_lags(in1.size, in2.size)
    lag[p-1] = lags[np.argmax(out)]
    fig,ax=plt.subplots(3)
    
    ax[0].plot(lags,out,'k')
    ax[1].plot(in1,'r')
    ax[2].plot(in2,'b')
    ax[0].set(xlabel=xlab, ylabel=ylab)
    
    ax[0].yaxis.set_major_formatter(FormatStrFormatter('%.2f'))

    ax[1].yaxis.set_major_formatter(FormatStrFormatter('%.2f'))

    ax[2].yaxis.set_major_formatter(FormatStrFormatter('%.2f'))

    fig.suptitle(f'Between {dist[index[i-1]].values} and {dist[index[i]].values} km')
    
fig.tight_layout()

In [ ]:
index=[0,46,99,149,207]
c=-((dist[index[:-1]].values-dist[index[1:]].values)*1000)/((time[lag.astype(int)]-2880)*60)

In [ ]:
c

In [ ]:
params = {'font.size': 28,
          'figure.figsize': (30, 15),
         'font.family':'serif'}
pl.rcParams.update(params)


In [ ]:
fig = plt.figure()
gs = GridSpec(nrows=1, ncols=2)#, height_ratios=[1,1])

lines=0
cbarall=0
Wmin=-0.000003
Wmax=0.000003
markers=Line2D.filled_markers
markers=np.delete(markers,np.arange(2,5,1))
colors=['b','g','r','k','m','c','y','brown','lime','fuchsia']

ax0 = fig.add_subplot(gs[0, 0])
plot_HOVMÖLLER(ax0,dist,TIME,Wfilt,'',
               'Vertical velocity [m/s]',Wmin,Wmax,fig,lat_ac,lon_ac,lines,cbarall)

p=0
index=[0,46,99,149,207]
for i in index:
    p=p+1
    ax0.plot(np.ones_like(dist[i])*TIME[288],dist[i],'o', 
            markersize=18, color=colors[p-1],marker=markers[p-1])
    ax0.axhline(y=dist[i],color=colors[p-1],marker=markers[p-1])
    if i!=207:
        ax0.annotate(f'{c[p-1]:.2f} m/s',(TIME[428],dist[i+5]))


ax3 = fig.add_subplot(gs[0,1])
ax3.set_facecolor('tan')
pc = ax3.contourf(LON,LAT,np.ma.masked_array(depth, mask=maskw[0,:,:]),50,
                 vmin=0, vmax=5000, cmap=cmocean.cm.deep)#, extend='max')
cb = plt.colorbar(pc, extend='max',label='depth / m')
cn = ax3.contour(LON,LAT,depth, colors=['0.3','0.6'], 
                levels=[250,500])


p=0
for i in index:
    p=p+1
    ax3.axhline(y=LAT[lat_ac[i]],color=colors[p-1],marker=markers[p-1])
#ax3.annotate('800 km from the bay',xy=(340,350), xytext=(340, 350+50),xycoords='axes points',fontsize=10)
p=0
for ii,jj in zip(lon_ac[index],lat_ac[index]):
    p=p+1
    ax3.plot(LON[ii-1],LAT[jj-1],marker=markers[p-1], 
            markersize=18, color=colors[p-1])


cb.set_label('Depth [m]')
ax3.set_xlabel('Lon [°]')
ax3.set_ylabel('Lat [°]')
ax3.set_xlim(238-360, 246-360)
ax3.set_ylim(27,35.3)
#ax3.set_aspect(1)
fig.tight_layout()

###### For the evenly spaced out distances

In [ ]:
lag=np.zeros(9)
p=0

xlab='Lag'
ylab='Correlation'

for i in np.arange(1,10,1):
    p=p+1
    index=np.arange(0,540,54)
    in1=Wfilt[:,index[i]]
    in2=Wfilt[:,index[i-1]]
    out=sig.correlate(in1, in2)
    lags = sig.correlation_lags(in1.size, in2.size)
    lag[p-1] = lags[np.argmax(out)]
    fig,ax=plt.subplots(3)
    
    ax[0].plot(lags,out,'k')
    ax[1].plot(in1,'r')
    ax[2].plot(in2,'b')
    ax[0].set(xlabel=xlab, ylabel=ylab)
    
    ax[0].yaxis.set_major_formatter(FormatStrFormatter('%.2f'))

    ax[1].yaxis.set_major_formatter(FormatStrFormatter('%.2f'))

    ax[2].yaxis.set_major_formatter(FormatStrFormatter('%.2f'))

    fig.suptitle(f'Between {dist[index[i-1]].values} and {dist[index[i]].values} km')
    
fig.tight_layout()

In [ ]:
index=np.arange(0,540,54)
c=-((dist[index[:-1]].values-dist[index[1:]].values)*1000)/((time[lag.astype(int)]-2880)*60)

In [ ]:
c

In [ ]:
fig = plt.figure()
gs = GridSpec(nrows=1, ncols=2)#, height_ratios=[1,1])

lines=0
cbarall=0
Wmin=-0.000003
Wmax=0.000003
markers=Line2D.filled_markers
markers=np.delete(markers,np.arange(2,5,1))
colors=['b','g','r','k','m','c','y','brown','lime','fuchsia']

ax0 = fig.add_subplot(gs[0, 0])
plot_HOVMÖLLER(ax0,dist,TIME,Wfilt,' ',
               'Vertical velocity [m/s]',Wmin,Wmax,fig,lat_ac,lon_ac,lines,cbarall)

p=0
index=np.arange(0,540,54) 
for i in index:
    p=p+1
    ax0.plot(np.ones_like(dist[i])*TIME[288],dist[i],'o', 
            markersize=18, color=colors[p-1],marker=markers[p-1])
    ax0.axhline(y=dist[i],color=colors[p-1],marker=markers[p-1])
    if i!=486:
        ax0.annotate(f'{c[p-1]:.2f} m/s',(TIME[428],dist[i+5]))


ax3 = fig.add_subplot(gs[0,1])
ax3.set_facecolor('tan')
pc = ax3.contourf(LON,LAT,np.ma.masked_array(depth, mask=maskw[0,:,:]),50,
                 vmin=0, vmax=5000, cmap=cmocean.cm.deep)#, extend='max')
cb = plt.colorbar(pc, extend='max',label='depth / m')
cn = ax3.contour(LON,LAT,depth, colors=['0.3','0.6'], 
                levels=[250,500])


p=0
for i in index:
    p=p+1
    ax3.axhline(y=LAT[lat_ac[i]],color=colors[p-1],marker=markers[p-1])
#ax3.annotate('800 km from the bay',xy=(340,350), xytext=(340, 350+50),xycoords='axes points',fontsize=10)
p=0
for ii,jj in zip(lon_ac[index],lat_ac[index]):
    p=p+1
    ax3.plot(LON[ii-1],LAT[jj-1],marker=markers[p-1], 
            markersize=18, color=colors[p-1])


cb.set_label('Depth [m]')
ax3.set_xlabel('Lon [°]')
ax3.set_ylabel('Lat [°]')
ax3.set_xlim(238-360, 246-360)
ax3.set_ylim(27,35.3)
#ax3.set_aspect(1)
fig.tight_layout()

# POWER DENSITY SPECTRA

In [ ]:
def FFRQ(Wdif,Wfilt,timemin,dist):

    times = (timemin)*60 #in s

    t0 = 172800 # start is day 2 
    dt = 1200 # 20 min 
    timeproject=14400*60
    freqlim = (1./dt)
        
    nx = len(dist)
    nt = int(len(times)/2)+1
        
    psd = np.zeros((nx,nt))*np.nan
    phase = np.zeros((nx,nt))*np.nan
    
    psdfilt = np.zeros((nx,nt))*np.nan
    phasefilt = np.zeros((nx,nt))*np.nan
    
    freq =  np.zeros((nx,nt))*np.nan
    freqfilt = np.zeros((nx,nt))*np.nan
    

    for ii in np.arange(nx): #nx
        signalFFT = np.fft.rfft(Wdif[:,ii])
        signalFFTfilt = np.fft.rfft(Wfilt[:,ii])
    
        ## Get Power Spectral Density
        signalPSD = np.abs(signalFFT)
        signalPSDfilt = np.abs(signalFFTfilt)
        #signalPSD /= len(signalFFT)**2

        ## Get Phase
        #signalPhase = np.angle(signalFFT)

        ## Get frequencies corresponding to signal 
        fftFreq = np.fft.rfftfreq(len(Wdif[:,ii]), dt)
        fftFreqfilt = np.fft.rfftfreq(len(Wfilt[:,ii]), dt)
        
        psd[ii,:] = signalPSD[:]
        psdfilt[ii,:] = signalPSDfilt[:]
        freq[ii,:] =  fftFreq[:]
        freqfilt[ii,:] = fftFreqfilt[:]
        #phase[ii,:] = signalPhase[:]
    
   # psd=psd[freq>=(1/timeproject)]
    #freq=freq[freq>=(1/timeproject)]
    #psd=psd[np.where(freq<=freqlim)]
    #freq=freq[freq<=freqlim]
    
   # psdfilt=psdfilt[freqfilt>=(1/timeproject)]
    #freqfilt=freqfilt[freqfilt>=(1/timeproject)]
    #psdfilt=psdfilt[freqfilt<=freqlim]
    #freqfilt=freqfilt[freqfilt<=freqlim]
        
        
    return psd, freq,psdfilt,freqfilt

In [ ]:
psd, freq,psdfilt,freqfilt=FFRQ(Wdif,Wfilt,TIME,dist) 

In [ ]:
params = {'font.size': 28,
          'figure.figsize': (15, 30),
         'font.family':'serif'}
pl.rcParams.update(params)

In [ ]:
timepsd=np.arange(2880,TIME[-1],40)
timepsd=np.append(timepsd,14400)
periodsplot=(1/freq[:,:])/(60*60*24)
periodsplotfilt=(1/freqfilt[:,:])/(60*60*24)

In [ ]:
fig, ax = plt.subplots(2)

vmin=0
vmax=0.0009
psdplot=np.transpose(psd[:,1:])
title='PSD [$ms^{-1}Hz^{-1}$]'
xlab='Time [days]'
ylab='Distance [km]'

#time= TIME.values.astype(int)
#time=dat.date2num(TIME)
markers=Line2D.filled_markers
markers=np.delete(markers,np.arange(2,5,1))
colors=['b','g','r','k','m','c','y','brown','lime','fuchsia']

ax[0].set(xlabel=xlab, ylabel=ylab,title='On unfiltered data')
ax[0].set_xticks([0, 1, 2, 3, 4, 5, 6, 7, 8])
ax[0].set_xticklabels([0, 1, 2, 3, 4, 5, 6, 7, 8])

#val=VAL.values
cax = ax[0].pcolormesh(periodsplot[1,1:],dist,psd[:,1:],vmin=vmin,vmax=vmax, cmap=cmocean.cm.balance)

ax[1].set(xlabel=xlab, ylabel=ylab,title='On filtered data')
ax[1].set_xticks([0, 1, 2, 3, 4, 5, 6, 7, 8])
ax[1].set_xticklabels([0, 1, 2, 3, 4, 5, 6, 7, 8])

#val=VAL.values
cax = ax[1].pcolormesh(periodsplotfilt[1,1:],dist,psdfilt[:,1:],vmin=vmin,vmax=vmax, cmap=cmocean.cm.balance)
#ax.xaxis.set_major_locator(dat.MonthLocator())

p=0
#index=[0,46,99,149,207]
index=np.arange(0,540,54) 
for i in index:
    p=p+1
    ax[1].axhline(y=dist[i],color=colors[p-1],marker=markers[p-1])
    if i!=index[-1]:
        
        
        locfilt=np.where(psdfilt[index[p-1]:index[p],:]==np.max(psdfilt[index[p-1]:index[p],:]))

        peakfilt=periodsplotfilt[locfilt]
        
        ax[1].plot(peakfilt,dist[locfilt[0]+i],'o', 
            markersize=18, color=colors[p-1],marker=markers[p-1])
        
        ax[1].annotate(f'Max PSD at {peakfilt[0]:.2f} days',(6,dist[i+3]),color='w')

p=0
#index=[0,46,99,149,207]
index=np.arange(0,540,54) 
for i in index:
    p=p+1
    ax[0].axhline(y=dist[i],color=colors[p-1],marker=markers[p-1])
    if i!=index[-1]:
        
        
        loc=np.where(psd[index[p-1]:index[p],:]==np.max(psd[index[p-1]:index[p],:]))

        peak=periodsplot[loc]
        
        ax[0].plot(peak,dist[loc[0]+i],'o', 
            markersize=18, color=colors[p-1],marker=markers[p-1])
        
        ax[0].annotate(f'Max PSD at {peak[0]:.2f} days',(6,dist[i+3]),color='w')

cbar_ax = fig.add_axes([1, 0.15, 0.03, 0.7])
fig.colorbar(cax, cax=cbar_ax)
cbar_ax.set_ylabel(title)


#print(f'For the nonfiltetered data max PSD is found at period {peak} days and at distance {peakdist} km from the bay')

#print(f'For the filtered data max PSD is found at period {peakfilt} days and at distance {peakfiltdist} km from the bay')

In [ ]:
def plotfreqdens(freq,psd,freqfilt,psdfilt):
    fig, ax = plt.subplots(2,2,figsize=(20,10))
    xlab='Frequency [Hz]'
    ylab='Power density spectra [W/Hz]'
    
    
    ax[0,0].plot(freq[0,freq[0,:]>0],psd[0,freq[0,:]>0])
    ax[1,0].plot(freqfilt[0,freqfilt[0,:]>0],psdfilt[0,freqfilt[0,:]>0])
    ax[0,1].plot(freq[-1,freq[-1,:]>0],psd[-1,freq[-1,:]>0])
    ax[1,1].plot(freqfilt[-1,freqfilt[-1,:]>0],psdfilt[-1,freqfilt[-1,:]>0])
    peak=freq[0,np.argmax(psd[0,freq[0,:]>0])]
    period=1/peak
    peakend=freq[-1,np.argmax(psd[-1,freq[-1,:]>0])]
    periodend=1/peakend

    peakfilt=freqfilt[0,np.argmax(psdfilt[0,freqfilt[0,:]>0])]
    periodfilt=1/peakfilt

    peakfiltend=freqfilt[-1,np.argmax(psdfilt[-1,freqfilt[-1,:]>0])]
    periodfiltend=1/peakfiltend

    title=('Nonfiltered data:\n\n Beginning of bay:  \n Period '+ str(period/(60*60*24)) + 
           ' days\n Peak frequency '+ str(peak)+' Hz')
    titlefilt=('Filtered data:\n\n Beginning of bay:  \n Period '+ str(periodfilt/(60*60*24))
               + ' days\n Peak frequency '+ str(peakfilt)+' Hz')
    titleend=('End of bay: \n Period '+ str(periodend/(60*60*24)) + 
              ' days\n Peak frequency '+ str(peakend)+' Hz')
    titlefiltend=('End of bay:\n Period '+ str(periodfiltend/(60*60*24))
                  + ' days\n Peak frequency '+ str(peakfiltend)+' Hz')

    ax[0,0].set(xlabel=xlab, ylabel=ylab,title=title)
    ax[1,0].set(xlabel=xlab, ylabel=ylab,title=titleend)

    ax[0,1].set(xlabel=xlab, ylabel=ylab,title=titlefilt)
    ax[1,1].set(xlabel=xlab, ylabel=ylab,title=titlefiltend)


In [ ]:
plotfreqdens(freq,psd,freqfilt,psdfilt)

# Changes in velocity over the whole basin
##### By specifying certain variables you can get a certain time step at a certain depth, at a certain latitude. Either all of these so you get one value or select one or two variables to be constant

In [ ]:
def unstagger(ugrid, vgrid,wgrid):
    """ Interpolate u and v component values to values at grid cell centres (from D.Latornell for NEMO output).
    The shapes of the returned arrays are 1 less than those of
    the input arrays in the y and x dimensions.
    :arg ugrid: u velocity component values with axes (..., y, x)
    :type ugrid: :py:class:`numpy.ndarray`
    :arg vgrid: v velocity component values with axes (..., y, x)
    :type vgrid: :py:class:`numpy.ndarray`
    :returns u, v: u and v component values at grid cell centres
    :rtype: 2-tuple of :py:class:`numpy.ndarray`
    """
    u = np.add(ugrid[..., :-1], ugrid[..., 1:]) / 2
    v = np.add(vgrid[..., :-1, :], vgrid[..., 1:, :]) / 2
    w = np.add(wgrid[:,:-1, :, :], wgrid[:,1:, :, :]) / 2
    return u, v,v

In [ ]:
def loadNetCDFs(varname):
    dsw=[]
    dsn=[]
    for i in np.arange(0,8,1):
        
        pathn='/media/amelia/Trillian/SVB/exp06_512x612x100_ORL/01b_noSVB_febTS/'+ str(varname)+'noSVB'+ str(2+i)+'_'+ str(3+i) +'.nc'
        pathw='/media/amelia/Trillian/SVB/exp06_512x612x100_ORL_SVB/01b_SVB_febTS/'+ str(varname)+'withSVB'+ str(2+i)+'_'+ str(3+i) +'.nc'
        
        dswin  = xr.open_dataset(pathw)
        dsnin = xr.open_dataset(pathn)
        
        dsw.append(dswin)
        dsn.append(dsnin)

    return dsw, dsn

def getvaluesNetCDfs(varname,ind,specifictime,atdepth,atlat,t,d,y):
    dsw,dsn=loadNetCDFs(str(varname))
    hFacC = dsn[ind].hFacC.values
    if specifictime==1:
        if atdepth==1:
            VALw=dsw[ind].WVEL[t,d,:,:].values
            VALn=dsn[ind].WVEL[t,d,:,:].values
            hfa = np.ma.masked_values(hFacC[d,:,:], 0)
            VAL=VALw-VALn
        elif atlat==1:
            VALw=dsw[ind].WVEL[t,:,y,:].values
            VALn=dsn[ind].WVEL[t,:,y,:].values
            hfa = np.ma.masked_values(hFacC[:,y,:], 0)
            VAL=VALw-VALn
    else:
        VALw,VALn=LOADVAR(varname,d)
        VAL=VALw-VALn
        if detrend==1:
            VALdif=sig.detrend(VAL,axis=0)
        else:
            VALdif=0                
        if filtering==1:
            VALfilt=np.zeros(np.shape(VALdif))
            order = 5
            fs = 1/1200       
            cutoff =np.array([1/432000, 1/43200])

            for d in np.arange(np.size(VALdif,1)):
                for k in np.arange(np.size(VALdif,2)):
                    data = VALdif[:,d,k]
                    VALfilt[:,d,k] = butter_lowpass_filter(data, cutoff, fs, order)
        else: 
            VALfilt=0
        
    TIME=dsw[ind].time.values.astype(int)*1e-9
    LON=dsw[ind].XC-360
    LAT=dsw[ind].YC
    Z=dsw[ind].Z
    mask = np.ma.getmask(hfa)
        
        
    return(VALw,VALn,TIME,LON,LAT,Z,mask,VAL,VALdif,VALfilt)

In [ ]:
def LOADVAR(varname,d):
    dsw,dsn=loadNetCDFs(varname)
    
    var23=dsw[0].WVEL[:,d,:,:].values
    var34=dsw[1].WVEL[:,d,:,:].values
    var45=dsw[2].WVEL[:,d,:,:].values
    var56=dsw[3].WVEL[:,d,:,:].values
    var67=dsw[4].WVEL[:,d,:,:].values
    var78=dsw[5].WVEL[:,d,:,:].values
    var89=dsw[6].WVEL[:,d,:,:].values
    var910=dsw[7].WVEL[:,d,:,:].values
    
    VALw=np.concatenate((var23, var34, var45, var56,var67,var78,var89, var910), axis=0) 
    
    var23n=dsn[0].WVEL[:,d,:,:].values
    var34n=dsn[1].WVEL[:,d,:,:].values
    var45n=dsn[2].WVEL[:,d,:,:].values
    var56n=dsn[3].WVEL[:,d,:,:].values                 
    var67n=dsn[4].WVEL[:,d,:,:].values
    var78n=dsn[5].WVEL[:,d,:,:].values
    var89n=dsn[6].WVEL[:,d,:,:].values
    var910n=dsn[7].WVEL[:,d,:,:].values
    
    VALn=np.concatenate((var23n, var34n, var45n, var56n,var67n,var78n,var89n, var910n), axis=0) 
    
    return VALw,VALn

# Visualising the whole basin

##### Plot the basin from above, at one depth, one time step

In [ ]:
def plot_above(ax,mask,LON,LAT, VALb,VALn,title,vmin,vmax,lat_fix,lon_fix,vfilt):
    
    xlab='Longitude [°]'
    ylab='Latitude [°]'
    
    
    ax.set_facecolor('wheat')
    cax = ax.pcolormesh(LON,LAT,np.ma.masked_array(VALb.values-VALn.values, mask=mask),cmap=cmocean.cm.balance,vmin=vmin,vmax=vmax)
    for ii,jj in zip(lon_fix,lat_fix):

        ax.plot(LON[ii-1],LAT[jj-1],'o', 
                markersize=10, color='r')
 
    asign=np.sign(vfilt)
    signchange = np.where(((np.roll(asign, 1) - asign) != 0))
        
    ax.plot(LON[lon_fix[signchange]],LAT[lat_fix[signchange]],'o', 
                markersize=10, color='b')
    ax.set_title(title)
    cbar_ax = plt.colorbar(cax, ax=ax)
    ax.set(xlabel=xlab, ylabel=ylab)
    cbar_ax.set_label('Vertical velocity [m/s]')
    ax.set_xlim(-122,-114) 
    ax.set_ylim(27,35.3)


In [ ]:
fig,ax=plt.subplots()
vmin=-0.000002
vmax=0.000002
dep=55
t=28
tt=72*ind+t

plot_above(ax,maskw[dep,:,:],LON,LAT,Wwall[t,dep,:,:],Wnall[t,dep,:,:],'Vertical velocity',
           vmin,vmax,lat_fix,lon_fix,Wfilt[tt,:])

## Animation GO TO ANIMATION SCRIPT TO SEE A BETTER REPRESENTATION

In [ ]:
ind=0
Wwall=dswall[ind].WVEL
Wnall=dsnall[ind].WVEL
TIME=dswall[ind].time.values.astype(int)
LON=dswall[ind].XC-360
LAT=dswall[ind].YC
times=TIME*1e-9
Z=dswall[ind].Zl

hFacCw = dsnall[ind].hFacC
hFacCusew=hFacCw.values

hfa = np.ma.masked_values(hFacCusew, 0)
maskw = np.ma.getmask(hfa)

In [ ]:
fig,ax=plt.subplots()
vmin=-0.000002
vmax=0.000002
dep=55
t=29

tt=(((72*ind+t)*20)+2880)/60
Win=Wwall[t,dep,:,:]-Wnall[t,dep,:,:]
plot_foranim(ax,maskw[dep,:,:],LON,LAT, Win,vmin,vmax,tt,Z,dep)

In [ ]:
def plot_foranim(ax,mask,LON,LAT, VAL,vmin,vmax,tt,Z,dep):
    
    xlab='Longitude [°]'
    ylab='Latitude [°]'
    
    
    ax.set_facecolor('wheat')
    cax = ax.pcolormesh(LON,LAT,np.ma.masked_array(VAL,mask=mask),cmap=cmocean.cm.balance,vmin=vmin,vmax=vmax)
    ax.set(xlabel=xlab, ylabel=ylab)
    ax.set_title(f'At depth {Z[dep].values:.2f}  m. After {tt:.1f} hours')
    ax.set_xlim(-122,-115.8)     
    cbar = plt.colorbar(cax)
    cbar.set_label('Vertical velocity [m/s]')
    ax.set_ylim(27,35.3)
    


In [ ]:
def get_snapshot_at_level(t,dep,dsw,dsn):
    ind=0
    if t>=72 and t <(72*2):
        ind=1
        t=t-72
    elif t>=(72*2) and t<(72*3):
        ind=2
        t=t-(72*2)
    elif t>=(72*3) and t<(72*4):
        ind=3
        t=t-(72*3)
    elif t>=(72*4) and t<(72*5):
        ind=4
        t=t-(72*4)
    elif t>=(72*5) and t<(72*6):
        ind=5
        t=t-(72*5)
    elif t>=(72*6) and t<(72*7):
        ind=6
        t=t-(72*6)
    elif t>=(72*7) and t<(72*8):
        ind=7
        t=t-(72*7)
    
    Ww=dsw[ind].WVEL[t,dep,:,:].values
    Wn=dsn[ind].WVEL[t,dep,:,:].values
    W = Ww-Wn
    return(W)

def animate(t):
    tt=(t*20+2880)/60
    dep=55
    vmin=-0.000002
    vmax=0.000002
    print(t)
    W = get_snapshot_at_level( t,dep,dswall,dsnall)
    

    cax.set_array(np.ma.masked_array(W,mask=maskw[dep,:,:]))
    ax.set_title(f'At depth {Z[dep].values:.2f} m. After {tt:.1f} hours')


In [ ]:
Writer = animation.writers['ffmpeg']
writer = Writer(fps=4, metadata=dict(artist='AM'), bitrate=2000)

In [ ]:
params = {'font.size': 28,
          'figure.figsize': (15, 15),
         'font.family':'serif'}
pl.rcParams.update(params)


In [ ]:
fig, ax = plt.subplots()

 #483.2 meter depth is the 55th element

vmin=-0.000002
vmax=0.000002
dep=55
t=0
#Index to call from the list of netcdfs
ind=0 #0 is day 2-3, 1 is day 3-4 until index 7 (day 9-10)
tt=(((72*ind+t)*20)+2880)/60 # Gives amount of hours from start of the model, starts at hour 48 if ind=0 and t=0
Wwall=dswall[ind].WVEL
Wnall=dsnall[ind].WVEL
Win=Wwall[t,dep,:,:]-Wnall[t,dep,:,:]




xlab='Longitude [°]'
ylab='Latitude [°]'

ax.set_facecolor('wheat')
cax = ax.pcolormesh(LON,LAT,np.ma.masked_array(Win,mask=maskw[dep,:,:]),
                    cmap=cmocean.cm.balance,vmin=vmin,vmax=vmax)
ax.set(xlabel=xlab, ylabel=ylab)

ax.set_title(f'At depth {Z[dep].values:.2f} m. After {tt:.1f} hours')
ax.set_xlim(-122,-115.8)     
cbar = plt.colorbar(cax)
cbar.set_label('Vertical velocity [m/s]')
ax.set_ylim(27,35.3)

anim = FuncAnimation(fig, animate,frames=500, repeat=False)

    
anim.save('WWELtest.mp4', writer=writer, dpi=600)


# When looking at all depths NOT WORKING WELL

#### Loading variables

In [ ]:
Z=dww[0].z
lon_fix=dww[0].lonAC
lat_fix=dww[0].latAC
X=dww[0].x
nt = np.size(time)

dist30=X[np.where(lat_fix==ind30)]
dist307=X[np.where(lat_fix==ind307)]
dist314=X[np.where(lat_fix==ind314)]
dist32=X[np.where(lat_fix==ind32)]

#### Filtering

In [ ]:
order = 5
fs = 1/1200       
cutoff =np.array([1/432000, 1/43200])

Wfilt=np.zeros(np.shape(Wdif))
for dep in np.arange(np.size(Wdif,1)):
    for d in np.arange(np.size(Wdif,2)):
        data = Wdif[:,dep,d]
        # Filtering and plotting
        Wfilt[:,dep,d] = butter_lowpass_filter(data, cutoff, fs, order)

Vfilt=np.zeros(np.shape(Vdif))
for dep in np.arange(np.size(Vdif,1)):
    for d in np.arange(np.size(Vdif,2)):
        data = Vdif[:,dep,d]
        # Filtering and plotting
        Vfilt[:,dep,d] = butter_lowpass_filter(data, cutoff, fs, order)
    
Ufilt=np.zeros(np.shape(Udif))
for dep in np.arange(np.size(Udif,1)):
    for d in np.arange(np.size(Udif,2)):
        data = Udif[:,dep,d]
        # Filtering and plotting
        Ufilt[:,dep,d] = butter_lowpass_filter(data, cutoff, fs, order)

In [ ]:
params = {'font.size': 28,
          'figure.figsize': (30, 30),
         'font.family':'serif'}
pl.rcParams.update(params)


#### Showing effect of the filter

In [ ]:
fig,ax=plt.subplots(6,2)

d=39
dd=6
xlab='Time (days)'
ylab='Vertical velocity (m/s)'
distance=X.values
VAL=Wfilt[:,dd,:]
VAL2=W[:,dd,:]
ylim=[-0.00001, 0.00001]
ylim2=[-0.0000075, 0.0000075]
ylim3=[-0.000005, 0.000005]
ylim4=[-0.0000025, 0.0000025]
ylim5=[-0.000001, 0.000001]

plt.setp(ax, xticks=[2880, 4320, 5760, 7200, 8640, 10080, 11520, 12960, 14400,], xticklabels=[2, 3, 4, 5, 6, 7, 8, 9, 10])

#ax[0,0].plot(time, VAL2[:,0],'r')
ax[0,0].plot(time, VAL[:,0])
ax[0,0].set(xlabel=xlab, ylabel=ylab)
ax[0,0].set_title('%1.1f km from the bay' %distance[d*0])
ax[0,0].set_ylim([-np.nanmax(VAL[:,0]),np.nanmax(VAL[:,0])])

#ax[0,1].plot(time, VAL2[:,d],'r')
ax[0,1].plot(time, VAL[:,d])
ax[0,1].set(xlabel=xlab, ylabel=ylab)
ax[0,1].set_title('%1.1f km from the bay' %distance[d*1])
ax[0,1].set_ylim([-np.nanmax(VAL[:,d]),np.nanmax(VAL[:,d])])

#ax[1,0].plot(time, VAL2[:,d*2],'r')
ax[1,0].plot(time, VAL[:,d*2])
ax[1,0].set(xlabel=xlab, ylabel=ylab)
ax[1,0].set_title('%1.1f km from the bay' %distance[d*2])
ax[1,0].set_ylim([-np.nanmax(VAL[:,d*2]),np.nanmax(VAL[:,d*2])])

#ax[1,1].plot(time, VAL2[:,d*3],'r')
ax[1,1].plot(time, VAL[:,d*3])
ax[1,1].set(xlabel=xlab, ylabel=ylab)
ax[1,1].set_title('%1.1f km from the bay' %distance[d*3])
ax[1,1].set_ylim([-np.nanmax(VAL[:,d*3]),np.nanmax(VAL[:,d*3])])

#ax[2,0].plot(time, VAL2[:,d*4],'r')
ax[2,0].plot(time, VAL[:,d*4])
ax[2,0].set(xlabel=xlab, ylabel=ylab)
ax[2,0].set_title('%1.1f km from the bay' %distance[d*4])
ax[2,0].set_ylim([-np.nanmax(VAL[:,d*4]),np.nanmax(VAL[:,d*4])])

#ax[2,1].plot(time, VAL2[:,d*5],'r')
ax[2,1].plot(time, VAL[:,d*5])
ax[2,1].set(xlabel=xlab, ylabel=ylab)
ax[2,1].set_title('%1.1f km from the bay' %distance[d*5])
ax[2,1].set_ylim([-np.nanmax(VAL[:,d*5]),np.nanmax(VAL[:,d*5])])

#ax[3,0].plot(time, VAL2[:,d*6],'r')
ax[3,0].plot(time, VAL[:,d*6])
ax[3,0].set(xlabel=xlab, ylabel=ylab)
ax[3,0].set_title('%1.1f km from the bay' %distance[d*6])
ax[3,0].set_ylim([-np.nanmax(VAL[:,d*6]),np.nanmax(VAL[:,d*6])])

#ax[3,1].plot(time, VAL2[:,d*7],'r')
ax[3,1].plot(time, VAL[:,d*7])
ax[3,1].set(xlabel=xlab, ylabel=ylab)
ax[3,1].set_title('%1.1f km from the bay' %distance[d*7])
ax[3,1].set_ylim([-np.nanmax(VAL[:,d*7]),np.nanmax(VAL[:,d*7])])

#ax[4,0].plot(time, VAL2[:,d*8],'r')
ax[4,0].plot(time, VAL[:,d*8])
ax[4,0].set(xlabel=xlab, ylabel=ylab)
ax[4,0].set_title('%1.1f km from the bay' %distance[d*8])
ax[4,0].set_ylim([-np.nanmax(VAL[:,d*8]),np.nanmax(VAL[:,d*8])])

#ax[4,1].plot(time, VAL2[:,d*9],'r')
ax[4,1].plot(time, VAL[:,d*9])
ax[4,1].set(xlabel=xlab, ylabel=ylab)
ax[4,1].set_title('%1.1f km from the bay' %distance[d*9])
ax[4,1].set_ylim([-np.nanmax(VAL[:,d*9]),np.nanmax(VAL[:,d*9])])

#ax[5,0].plot(time, VAL2[:,d*10],'r')
ax[5,0].plot(time, VAL[:,d*10])
ax[5,0].set(xlabel=xlab, ylabel=ylab)
ax[5,0].set_title('%1.1f km from the bay' %distance[d*10])
ax[5,0].set_ylim([-np.nanmax(VAL[:,d*10]),np.nanmax(VAL[:,d*10])])

#ax[5,1].plot(time, VAL2[:,d*11],'r')
ax[5,1].plot(time, VAL[:,d*11])
ax[5,1].set(xlabel=xlab, ylabel=ylab)
ax[5,1].set_title('%1.1f km from the bay' %distance[d*11])
ax[5,1].set_ylim([-np.nanmax(VAL[:,d*11]),np.nanmax(VAL[:,d*11])])

fig.tight_layout()

#for i in np.arange(0,12,1):

 #   Wderiv=(Wfilt[:-1,dd,d*i]-Wfilt[1:,dd,d*i])/(time[:-1]-time[1:])

  #  asign=np.sign(Wderiv)
   # signchange = np.where(((np.roll(asign, 1) - asign) != 0))

   # timemax=time[signchange]
   # perpre=(timemax[2:]-timemax[:-2])/(60*24)
    #per=mean(perpre[perpre>=0.8])
    
    #print('%1.1f km from the bay' %distance[d*i])
    #print(f'Mean period after removing anomalies is {per:.3f} days ')


#### Hovmöller plot of vertical velocity in progress of filtering

In [ ]:
fig = plt.figure(figsize=(30, 30))
gs = GridSpec(nrows=3, ncols=1, height_ratios=[1, 1,1])


lines=1
cbarall=1

latmult=lat_fix.values
latarray=np.ones_like(W)*latmult[None,:]
lonmult=lon_fix.values
lonarray=np.ones_like(W)*lonmult[None,:]
maskpre=np.logical_or((lonarray >= 137), (latarray >= 552))
mask=maskpre==False
l=6
vmax=5e-7
vmin=-5e-7


ax0 = fig.add_subplot(gs[0, 0])
plot_HOVMÖLLER(ax0,X,time,W[:,l,:],'Vertical velocity raw data',
               'Vertical velocity (m/s)',vmin,vmax,fig,lat_fix,lon_fix,lines,cbarall)
ax1 = fig.add_subplot(gs[1, 0])
plot_HOVMÖLLER(ax1,X,time,Wdif[:,l,:],'Vertical velocity detrended',
               'Vertical velocity (m/s)',vmin,vmax,fig,lat_fix,lon_fix,lines,cbarall)
ax2 = fig.add_subplot(gs[2, 0])
plot_HOVMÖLLER(ax2,X,time,Wfilt[:,l,:],'Vertical velocity filtered',
               'Vertical velocity (m/s)',vmin,vmax,fig,lat_fix,lon_fix,lines,cbarall)


fig.tight_layout()

In [ ]:
fig = plt.figure(figsize=(40, 30))
gs = GridSpec(nrows=2, ncols=3, height_ratios=[1,1])

lines=1
cbarall=0

Wmin=-0.0000003
Wmax=0.0000003
Vmin=-0.0003
Vmax=0.0003
Umin=-0.0003
Umax=0.0003
latmult=lat_fix.values
latarray=np.ones_like(W)*latmult[None,:]
lonmult=lon_fix.values
lonarray=np.ones_like(W)*lonmult[None,:]
#maskpre=np.logical_or(lonarray >= 190, latarray >= 552)
#mask=maskpre==False
l=1

ax0 = fig.add_subplot(gs[0, 0])
plot_HOVMÖLLER(ax0,X,time,Wfilt[:,l,:],'Vertical velocity (m/s)',
               'Vertical velocity (m/s)',Wmin,Wmax,fig,lat_fix,lon_fix,lines,cbarall)
ax1 = fig.add_subplot(gs[0, 1])
plot_HOVMÖLLER(ax1,X,time,Vfilt[:,l,:],'Alongshore velocity (m/s)',
               'Alongshore velocity (m/s)',Vmin,Vmax,fig,lat_fix,lon_fix,lines,cbarall) 
ax2 = fig.add_subplot(gs[0, 2])
plot_HOVMÖLLER(ax2,X,time,Ufilt[:,l,:],'Crosshore velocity (m/s)',
               'Crosshore velocity (m/s)',Umin,Umax,fig,lat_fix,lon_fix,lines,cbarall)

ax3 = fig.add_subplot(gs[1,0:2])
ax3.set_facecolor('tan')
pc = ax3.contourf(LON,LAT,np.ma.masked_array(depth, mask=maskw[0,:,:]),50,
                 vmin=0, vmax=5000, cmap=cmocean.cm.deep)#, extend='max')
cb = plt.colorbar(pc, extend='max',label='depth / m')
cn = ax3.contour(LON,LAT,depth, colors=['0.3','0.6'], 
                levels=[250,500])
ax3.axhline(y=LAT[518],color='k')
#ax3.annotate('800 km from the bay',xy=(340,350), xytext=(340, 350+50),xycoords='axes points',fontsize=10

for ii,jj in zip(lon_fix,lat_fix):

    ax3.plot(LON[ii-1],LAT[jj-1],'o', 
            markersize=4, color='r')

ax3.plot(LON[lon_fix[0]],LAT[lat_fix[0]],'o', 
        markersize=4, color='b')
ax3.plot(LON[lon_fix[404]],LAT[lat_fix[404]],'^', 
        markersize=4, color='b')


cb.set_label('Depth (m)')
ax3.set_xlabel('Lon [°]')
ax3.set_ylabel('Lat [°]')
ax3.set_xlim(238-360, 246-360)
ax3.set_ylim(27,35.3)
ax3.set_aspect(1)
fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,9))
ax.set_facecolor('tan')
pc = ax.contourf(LON,LAT,np.ma.masked_array(depth, mask=maskw[0,:,:]),50,
                 vmin=0, vmax=5000, cmap=cmocean.cm.deep)#, extend='max')
cb = plt.colorbar(pc, extend='max',label='depth / m')
cn = ax.contour(LON,LAT,depth, colors=['0.3','0.6'], 
                levels=[250,500])
ax.axhline(y=LAT[550],color='k')
ax.annotate('800 km from the bay',xy=(340,350), xytext=(340, 350+50),xycoords='axes points',fontsize=10)

#ax.axhline(y=31.4,color='k')
#ax.axhline(y=30.7,color='k')
#ax.axhline(y=30,color='k')

#for ii,jj in zip(lon_fix,lat_fix):
index=np.arange(0,540,54) 
ax.plot(LON[lon_fix],LAT[lat_fix],'o', 
            markersize=4, color='r')
    #print('Depth at cell is %1.2f m' % (depth[jj-1,ii-1]))

ax.plot(LON[lon_fix[index]],LAT[lat_fix[index]],'o', 
            markersize=4, color='k')
#for ii,jjin zip(lon_inds_2,lat_inds_2):
#    ax.plot(LON[ii],LAT[jj-1],'o', 
#            markersize=4, color='y')

ax.plot(LON[lon_fix[45]],LAT[lat_fix[45]],'o', 
        markersize=4, color='b')
ax.plot(LON[lon_fix[108]],LAT[lat_fix[108]],'o', 
        markersize=4, color='b')
#ax.plot(LON[lon_fix[321]],LAT[lat_fix[321]],'^', 
 #       markersize=4, color='b')
#ax.plot(LON[lon_fix[]],LAT[lat_fix[0]],'o', 
 #       markersize=4, color='b')
#ax.plot(LON[lon_fix[0]],LAT[lat_fix[0]],'o', 
 #       markersize=4, color='b')


cb.set_label('Depth (m)')
ax.set_xlabel('Lon [°]')
ax.set_ylabel('Lat [°]')
ax.set_xlim(238-360, 246-360)
ax.set_ylim(27,35.3)
ax.set_aspect(1)